# Period Module

## Initialisation

Import modules, etc

In [1]:
import os
import operator
import jinja2

from datetime import datetime
from PIL import Image

import unittest

from common import Printable, testExit, projdir

from fuzzy import FuzzyMatch
from entrant import Entrant

from constants import *

## Period Class

The period class contains common methods for events, sessions and courses

In [2]:
class Period(Printable):
    def __init__(self, parent=None, verbosity=1):
        '''Initialise period object'''

        super().__init__(verbosity=verbosity)

        self.period = self.__class__.__name__
        self.parent = parent

        if parent:
            self.year = parent.year
            self.entrants = parent.entrants
            self.names = parent.names
            self.sailNos = parent.sailNos
            self.gt31Ids = parent.gt31Ids
            self.fuzzyMatch = parent.fuzzyMatch
            self.appConfig = parent.appConfig
            self.eventConfig = parent.eventConfig
            self.reports = parent.reports

        else:
            self.year = None
            self.entrants = {}
            self.names = {}
            self.sailNos = {}
            self.gt31Ids = {}
            self.fuzzyMatch = FuzzyMatch()
            self.appConfig = None
            self.eventConfig = None
            self.reports = None

        self.runs = {}
        self.numRuns = 0


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        entrants = []

        for entrantId in self.runs:
            # Sort the runs for the individual entrant
            runs = self.runs[entrantId]
            runs.sort(key=lambda x: x.data[T_SPEED], reverse=True)
            
            # Period sort will be based on speed descending (hence the negative) and date/time ascending
            bestRun = runs[0]
            entrants.append((entrantId, runs, -bestRun.speed, bestRun.date, bestRun.time))

        # Sort avoids using a lambda function call
        entrants.sort(key = operator.itemgetter(2, 3, 4))
        
        # Re-create self.runs so that it is correctly sorted (Python 3.7 upwards)
        self.runs = {}
        for entrant in entrants:
            self.runs[entrant[0]] = entrant[1]


    def getEventName(self):
        '''Get the event name from event config or app config'''

        eventName = None

        if 'Event' in self.eventConfig:
            if 'Name' in self.eventConfig['Event']:
                eventName = self.eventConfig['Event']['Name']

        if eventName is None and 'Event' in self.appConfig:
            if 'Name' in self.appConfig['Event']:
                eventName = self.appConfig['Event']['Name']

        if eventName is None:
            eventName = 'Weymouth Speed Week'

        return eventName


    def getFastest(self):
        '''Get fastest run of the period'''

        if len(self.runs) > 0:
            entrantId = [*self.runs.keys()][0]
            run = self.runs[entrantId][0]

            fastest = '{} ({})'.format(run.entrant.getName(), run.entrant.getCraftType())

            if 'Decimals' in self.eventConfig:
                decimals = self.eventConfig['Decimals']
            else:
                decimals = 2

            if decimals == 3:
                speed = '{:.3f}'.format(run.speed)
            else:
                speed = '{:.2f}'.format(run.speed)

        else:
            fastest = 'No runs'

            speed = None

        return fastest, speed


    def runReports(self):
        '''Run all of the reports results'''

        htmlDir = os.path.join(projdir, 'docs', 'results', str(self.year))
        rootPath = os.path.join('..', '..')

        className = self.__class__.__name__
        if className == 'Session':
            htmlDir = os.path.join(htmlDir, self.date)
            rootPath = os.path.join(rootPath, '..')
        if not os.path.exists(htmlDir):
            os.makedirs(htmlDir)

        bannerFn = self.eventConfig['Event']['Banner']
        banner = {}

        banner['src'] = os.path.join(rootPath, 'img', bannerFn)
        banner['alt'] = self.eventConfig['Event']['Name']
        
        with Image.open(os.path.join(projdir, 'docs', 'img', bannerFn)) as bannerImage:
            banner['width'], banner['height'] = bannerImage.size

        files = []
        numWarnings = 0
        
        # Note: It is slightly debatable whether this class should interact with the Reports class at such a low level
        if className in self.reports.periods:
            reportSets = self.reports.periods[className]

            templatePath = os.path.join(projdir, 'python', 'templates')
            templateLoader = jinja2.FileSystemLoader(searchpath=templatePath)
            templateEnv = jinja2.Environment(loader=templateLoader, autoescape=True, trim_blocks=True, lstrip_blocks=True)

            eventName = self.getEventName()
            if className == 'Session':
                periodDate = datetime.strptime(self.date, "%Y%m%d").strftime("%a %-d %b")
                pageComment = self.comment
            else:
                periodDate = self.year
                pageComment = None

            for reportSet in reportSets.keys():
                reports = reportSets[reportSet]
                numResults = 0

                for report in reports:
                    report.prepareResults(self.runs)
                    numResults += len(report.results)

                    if report.warning and len(report.results) > 0:
                        numWarnings += 1

                htmlFile = os.path.join(htmlDir, reportSet + '.html')

                pageTitle = reports[0].suptitle
                pageDescription = pageTitle
                cssPath = os.path.join(rootPath, 'css')
                
                if numResults > 0:
                    template = templateEnv.get_template("results.html")
                    html = template.render(
                                    eventName=eventName,
                                    banner=banner,
                                    className=className,
                                    pageTitle=pageTitle,
                                    pageComment=pageComment,
                                    pageDescription=pageDescription,
                                    periodDate=periodDate,
                                    cssPath=cssPath,
                                    reports=reports)

                    with open(htmlFile, 'w', encoding='utf-8') as f:
                        f.write(html)
                    
                    file = {'id': reportSet, 'href': os.path.basename(htmlFile), 'text': pageTitle}
                    if reportSet not in self.appConfig['Priorities']:
                        self.appConfig['Priorities'][file] = 10
                    files.append(file)

                else:
                    try:
                        os.remove(htmlFile)
                    except OSError:
                        pass

        htmlFile = os.path.join(htmlDir, 'index.html')

        if len(files) > 0 or pageComment:
            subDirs = []
            if className == 'Session':
                periodDate = datetime.strptime(self.date, "%Y%m%d").strftime("%A %-d %B")
                pageTitle = '{}'.format(periodDate)
            else:
                pageTitle = 'Event Reports'
                for sessionId in reversed(self.sessions):
                    session = self.sessions[sessionId]

                    href = os.path.join(sessionId, 'index.html')
                    sessionTitle = datetime.strptime(sessionId, "%Y%m%d").strftime("%A %-d %B")

                    fastest,speed = session.getFastest()

                    subDir = {'id': sessionId, 'href': href, 'label': sessionTitle, 'fastest': fastest, 'speed': speed}
                    subDirs.append(subDir)
                    
            pageDescription=pageTitle

            files = sorted(files, key = lambda x: self.appConfig['Priorities'][x['id']])
            
            template = templateEnv.get_template("index.html")
            html = template.render(
                            eventName=eventName,
                            banner=banner,
                            className=className,
                            pageTitle=pageTitle,
                            pageComment=pageComment,
                            pageDescription=pageDescription,
                            periodDate=periodDate,
                            subDirTitle = 'Session Reports',
                            cssPath=os.path.join(rootPath, 'css'),
                            files=files,
                            subDirs=subDirs)

            with open(htmlFile, 'w', encoding='utf-8') as f:
                f.write(html)
        else:
            try:
                os.remove(htmlFile)
            except OSError:
                pass

        if numWarnings > 0:
            if className == 'Session':
                self.logWarning('{} report on {} generated results - {}'.format(
                    className, self.date, report.filter))
            else:
                self.logWarning('{} report generated results - {}'.format(
                    className, report.filter))

## Unit Tests

A handful of very basic tests, including a dummy session class

In [3]:
class TestPeriod(unittest.TestCase):
    '''Class to test Period class'''
    
    def testDummy(self, session=None):
        '''Test using dummy data'''

        pass

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## All Done!